# The environment

The next three code cells contains commands that will configure your Coogle Colab environment.

When you transfer the strategy to Quantiacs, remove these cells.

They are not relevant for the Quantiacs platform.

At first, setup the toolbox from github using pip:

In [1]:
###DEBUG###

! pip install git+https://github.com/quantiacs/toolbox.git 2>/dev/null

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

  Cloning https://github.com/quantiacs/toolbox.git to /tmp/pip-req-build-_f0717bz
  Resolved https://github.com/quantiacs/toolbox.git to commit 272e66e017d3eb6d40517ffa39cd6a92dc5072d8
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 9.7 MB/s eta 0:00:00
  Created wheel for qnt: filename=qnt-0.0.407-py3-none-any.whl size=115361 sha256=17edf824687ec72b70fbea8660aa10d76994b8fbaac692ac575222c0ddd589df
  Stored in directory: /tmp/pip-ephem-wheel-cache-aouwdn72/wheels/ca/03/48/1dbd01eee19642348a025488a55e2290c63d6b83473b2b8b9a
Successfully built qnt
  Attempti

<IPython.core.display.Javascript object>

Then install TA-Lib (indicators library) if you need it.

Instead of TA-Lib you can use qnt.ta or another library. In this case, skip the next cell.

Note that the installation can take several minutes.

In [2]:
"""
###DEBUG###
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
! make
!make install
!pip install Ta-Lib

# test import
import talib

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))
"""

"\n###DEBUG###\n!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz\n!tar -xzvf ta-lib-0.4.0-src.tar.gz\n%cd ta-lib\n!./configure --prefix=/usr\n! make\n!make install\n!pip install Ta-Lib\n\n# test import\nimport talib\n\n# decrease height\nfrom IPython.display import Javascript\ndisplay(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))\n"

Finally specify the api key and disable interactive charts.

You can find the api key in [your profile](https://quantiacs.com/personalpage/homepage).

We disable interactive charts in the library because interact+plotly do not work correctly in Google Colab.

In [3]:
###DEBUG###
import os

os.environ['API_KEY'] = "aa39740a-02b3-4dbb-a6b5-b871631d4ccc"
os.environ['NONINTERACT'] = 'True'

# The strategy

The next cell contains the strategy code itself.

In [4]:
%%javascript
window.IPython && (IPython.OutputArea.prototype._should_scroll = function(lines) { return false; })
// run this cell for disabling widget scrolling

<IPython.core.display.Javascript object>

In [5]:
import logging

import xarray as xr  # xarray for data manipulation

import qnt.data as qndata     # functions for loading data
import qnt.backtester as qnbt # built-in backtester
import qnt.ta as qnta         # technical analysis library
import qnt.stats as qnstats   # statistical functions

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

np.seterr(divide = "ignore")

from qnt.ta.macd import macd
from qnt.ta.rsi  import rsi
from qnt.ta.stochastic import stochastic_k, stochastic, slow_stochastic

from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier

NOTICE: The environment variable DATA_BASE_URL was not specified. The default value is 'https://data-api.quantiacs.io/'
NOTICE: The environment variable CACHE_RETENTION was not specified. The default value is '7'
NOTICE: The environment variable CACHE_DIR was not specified. The default value is 'data-cache'


In [6]:
#stock_data = qndata.stocks.load_spx_data(tail = 365 * 5, assets = ['NAS:AAL','NAS:AAPL','NAS:EA','NAS:CEPH','NAS:AKAM','NAS:DXCM','NAS:CA','NAS:ALTR~1','NAS:TLAB','NAS:FANG','NAS:GEN','NAS:BMC','NAS:SNPS','NAS:SBAC','NAS:TXN','NAS:PTC','NAS:BKR','NAS:EXC','NAS:ALGN','NAS:LKQ','NAS:ENPH','NAS:CCEP','NAS:ALTR','NAS:FOSL','NAS:HST'])
stock_data = qndata.stocks.load_spx_data(tail = 365 * 5, assets = ['NAS:AAL','NAS:AAPL'])

| | #                                             | 15975 Elapsed Time: 0:00:00
| | #                                             | 15975 Elapsed Time: 0:00:00
| |  #                                            | 45926 Elapsed Time: 0:00:00


fetched chunk 1/1 2s
Data loaded 4s


In [7]:
def get_features(data):
    """Enhanced feature construction using multiple technical indicators."""

    # MACD and its components
    macd, macd_signal, macd_histogram = qnta.macd(data.sel(field="close"))

    # Logarithmic transformation of the closing price
    price = data.sel(field="close").ffill("time").bfill("time").fillna(0)
    log_price = np.log(price)

    # Relative Strength Index (RSI)
    rsi = qnta.rsi(data.sel(field="close"))

    # trend:
    trend = qnta.roc(qnta.lwma(data.sel(field="close"), 60), 1)

    # volatility:
    volatility = qnta.tr(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"))
    volatility = volatility / data.sel(field="close")
    volatility = qnta.lwma(volatility, 14)

    # Combine all features
    result = xr.concat(
        [macd_signal, macd_histogram, log_price, rsi, trend, volatility],
        pd.Index(
            ["macd_signal", "macd_histogram", "log_price", "rsi", "trend", "volatility"],
            name="field"
        )
    )

    return result.transpose("time", "field", "asset")


In [8]:
my_features = get_features(stock_data)
display(my_features.sel(field="trend").to_pandas())

asset,NAS:AAL,NAS:AAPL
time,,
2020-01-22,NaN,NaN
2020-01-23,NaN,NaN
2020-01-24,NaN,NaN
2020-01-27,NaN,NaN
2020-01-28,NaN,NaN
...,...,...
2025-01-10,0.678094,-0.011361
2025-01-13,0.493014,-0.045787
2025-01-14,0.595151,-0.061104


In [9]:
def get_target_classes(data):
    """Clases objetivo para predecir si el precio sube o baja."""

    price_current = data.sel(field="close")
    price_future = qnta.shift(price_current, -1)

    class_positive = 1
    class_negative = 0

    target_price_up = xr.where(
        price_future > price_current, class_positive, class_negative
    )

    return target_price_up

In [10]:
# displaying the target classes:
my_targetclass = get_target_classes(stock_data)
display(my_targetclass.to_pandas())

asset,NAS:AAL,NAS:AAPL
time,,
2020-01-22,1,1
2020-01-23,0,0
2020-01-24,0,0
2020-01-27,1,1
2020-01-28,0,1
...,...,...
2025-01-10,0,0
2025-01-13,1,0
2025-01-14,0,1


In [11]:
def get_model():
    """Modelo de Random Forest."""
    return RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

In [12]:
# Create and train the models working on an asset-by-asset basis.

asset_name_all = stock_data.coords["asset"].values

models = dict()

for asset_name in asset_name_all:
    target_cur = my_targetclass.sel(asset=asset_name).dropna(dim="time", how="any")
    features_cur = my_features.sel(asset=asset_name).dropna(dim="time", how="any")

    target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

    if len(features_cur.time) < 10:
        continue  # Not enough data for training

    model = get_model()

    try:
        model.fit(feature_for_learn_df.values, target_for_learn_df)
        models[asset_name] = model
    except ValueError as e:
        logging.exception(f"ValueError occurred while training model for {asset_name}: {e}")
    except Exception as e:
        logging.exception(f"Error occurred while training model for {asset_name}: {e}")

print(models)

{'NAS:AAL': RandomForestClassifier(max_depth=5, random_state=42), 'NAS:AAPL': RandomForestClassifier(max_depth=5, random_state=42)}


In [13]:
# Performs prediction and generates output weights:

asset_name_all = stock_data.coords["asset"].values
weights = xr.zeros_like(stock_data.sel(field="close"))

for asset_name in asset_name_all:
    if asset_name in models:
        model = models[asset_name]
        features_all = my_features
        features_cur = features_all.sel(asset=asset_name).dropna(dim="time", how="any")
        if len(features_cur.time) < 1:
            continue
        try:
            weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)
        except KeyboardInterrupt as e:
            raise e
        except:
            logging.exception("model prediction failed")

print(weights)

<xarray.DataArray 'stocks_s&p500' (time: 1255, asset: 2)> Size: 20kB
array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 1.],
       [0., 0.],
       [0., 0.]])
Coordinates:
  * time     (time) datetime64[ns] 10kB 2020-01-22 2020-01-23 ... 2025-01-16
    field    <U5 20B 'close'
  * asset    (asset) <U8 64B 'NAS:AAL' 'NAS:AAPL'


In [14]:
def get_sharpe(stock_data, weights):
    """Calculates the Sharpe ratio"""
    rr = qnstats.calc_relative_return(stock_data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(stock_data, weights)
sharpe

5.655934173384878

In [15]:
def train_model(data):
    """Entrenar modelos Random Forest por activo."""

    asset_name_all = data.coords["asset"].values
    features_all = get_features(data)
    target_all = get_target_classes(data)

    models = dict()

    for asset_name in asset_name_all:
        target_cur = target_all.sel(asset=asset_name).dropna(dim="time", how="any")
        features_cur = features_all.sel(asset=asset_name).dropna(dim="time", how="any")

        target_for_learn_df, feature_for_learn_df = xr.align(
            target_cur, features_cur, join="inner"
        )

        if len(features_cur.time) < 10:
            continue  # No hay suficientes datos para entrenar

        model = get_model()

        try:
            model.fit(feature_for_learn_df.values, target_for_learn_df)
            models[asset_name] = model
        except ValueError as e:
            logging.exception(f"Error de valor al entrenar el modelo para {asset_name}: {e}")
        except Exception as e:
            logging.exception(f"Error al entrenar el modelo para {asset_name}: {e}")

    return models

In [16]:
def predict_weights(models, data):
    """Predicción de pesos usando los modelos."""

    asset_name_all = data.coords["asset"].values
    weights = xr.zeros_like(data.sel(field="close"))

    for asset_name in asset_name_all:
        if asset_name in models:
            model = models[asset_name]
            features_all = get_features(data)
            features_cur = features_all.sel(asset=asset_name).dropna(dim="time", how="any")

            if len(features_cur.time) < 1:
                continue

            try:
                weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = (
                    model.predict(features_cur.values)
                )
            except KeyboardInterrupt as e:
                raise e
            except Exception as e:
                logging.exception(f"Error al predecir los pesos para {asset_name}: {e}")

    return weights

In [17]:
# Calculate weights using the backtester:
weights = qnbt.backtest_ml(
    train                         = train_model,
    predict                       = predict_weights,
    train_period                  =  2 *365,  # the data length for training in calendar days
    retrain_interval              = 10 *365,  # how often we have to retrain models (calendar days)
    retrain_interval_after_submit = 1,        # how often retrain models after submission during evaluation (calendar days)
    predict_each_day              = False,    # Is it necessary to call prediction for every day during backtesting?
                                              # Set it to True if you suspect that get_features is looking forward.
    competition_type              = "stocks_s&p500",  # competition type
    lookback_period               = 365,                 # how many calendar days are needed by the predict function to generate the output
    start_date                    = "2005-01-01",        # backtest start date
    analyze                       = True,
    build_plots                   = True  # do you need the chart?
)

Run the last iteration...


| |#                                              | 15975 Elapsed Time: 0:00:00
| |            #                                | 3756885 Elapsed Time: 0:00:01


fetched chunk 1/2 9s


| |       #                                      | 985399 Elapsed Time: 0:00:00


fetched chunk 2/2 12s
Data loaded 13s


...

asset,NAS:AAL,NAS:AAPL,NAS:ABNB,NAS:ACGL,NAS:ADBE,NAS:ADI,NAS:ADP,NAS:ADSK,NAS:AEP,NAS:AKAM
time,,,,,,,,,,
2025-01-02,0.0,0.006329,0.006329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006329
2025-01-03,0.0,0.005780,0.005780,0.005780,0.005780,0.000000,0.000000,0.000000,0.000000,0.005780
2025-01-06,0.0,0.005025,0.005025,0.000000,0.000000,0.000000,0.005025,0.000000,0.005025,0.005025
2025-01-07,0.0,0.004505,0.004505,0.000000,0.004505,0.000000,0.004505,0.000000,0.000000,0.004505
2025-01-08,0.0,0.000000,0.004950,0.004950,0.000000,0.000000,0.004950,0.000000,0.000000,0.004950
2025-01-10,0.0,0.000000,0.003968,0.000000,0.003968,0.003968,0.003968,0.000000,0.003968,0.003968
2025-01-13,0.0,0.000000,0.004149,0.000000,0.004149,0.000000,0.004149,0.000000,0.004149,0.004149
2025-01-14,0.0,0.000000,0.004444,0.004444,0.004444,0.004444,0.000000,0.000000,0.004444,0.004444
2025-01-15,0.0,0.000000,0.004367,0.004367,0.000000,0.000000,0.000000,0.004367,0.004367,0.004367


Stats:


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2025-01-02,2.582416,-0.004117,0.213042,-0.074082,-0.619445,0.228306,0.048639,1.0,480.0,0.329041,5.703236
2025-01-03,2.610179,0.010751,0.213033,-0.064127,-0.619445,0.230904,0.049190,1.0,482.0,0.329108,5.701718
2025-01-06,2.605773,-0.001688,0.213013,-0.065707,-0.619445,0.230463,0.049091,1.0,485.0,0.329181,5.700207
2025-01-07,2.597949,-0.003003,0.212993,-0.068512,-0.619445,0.229696,0.048924,1.0,489.0,0.329233,5.698984
2025-01-08,2.600839,0.001112,0.212972,-0.067476,-0.619445,0.229946,0.048972,1.0,489.0,0.329263,5.698201
2025-01-10,2.554431,-0.017843,0.212989,-0.084116,-0.619445,0.225448,0.048018,1.0,492.0,0.329304,5.697161
2025-01-13,2.576510,0.008644,0.212976,-0.076199,-0.619445,0.227534,0.048459,1.0,495.0,0.329359,5.696280
2025-01-14,2.594644,0.007038,0.212960,-0.069697,-0.619445,0.229231,0.048817,1.0,497.0,0.329380,5.695474
2025-01-15,2.616472,0.008413,0.212947,-0.061871,-0.619445,0.231262,0.049246,1.0,498.0,0.329405,5.694677


---


100% (5043 of 5043) |####################| Elapsed Time: 0:20:16 Time:  0:20:16
